> I had every intention to prepare for this technical talk.
> Sadly, that didn't work out because I faced these challenges.
> I was caught up in a heavy workload, and didn't have time to think about anything.
>  -- The Excuse Generator

# Transformers and Text Generation using GPT-2
We'll look at using the huggingface *transformers* library to generate text and perform some other natural language processing tasks.

The culmination of this NLP extravaganza will be a versitile, tireless, endlessly creative **Excuse Generator** powered by Artificial Intelligence.

```python
goal = 'write a better summary for this document'
tasks = ['contemplate the most important ideas', 'determine the most efficient way to summarize the ideas',
	 'summarize ways to apply to industry', 'discuss next steps']
s = ExcuseSituation(gen, assignment=goal, tasks=tasks, is_team=False, blame_others=False)
s.generate_excuses(count=1)
```

> I was supposed to write a better summary for this document.
> Things were going smoothly, but the problems started when I began to contemplate the most important ideas.
> I wanted to do this, but I couldn't because I would have to spend hours writing down all the details. 
-- The Excuse Generator

In [2]:
import random
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

## Resources

* [github.com/huggingface/transformers](https://github.com/huggingface/transformers#quick-tour)
* [Understanding GPT-2](https://towardsdatascience.com/openai-gpt-2-understanding-language-generation-through-visualization-8252f683b2f8)
* [Transformer Neural Network Architecture](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html) (Google Blog)
* [OpenAI: Better Language Models](https://openai.com/blog/better-language-models/)

## What is GPT-2

* Generative Pre-trained [Transformer](http://jalammar.github.io/illustrated-transformer/) from [OpenAI](https://openai.com/)
* Trained on 40GB of text from 8 million web pages
* Goal: predict the next word

### Sample of what GPT-2 can do

From the [OpenAI Blog](https://openai.com/blog/better-language-models/).

**Given the prompt:**
> In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

**GPT-2 Wrote**
>The scientist named the population, after their distinctive horn, Ovid’s Unicorn. These four-horned, silver-white unicorns were previously unknown to science.

>Now, after almost two centuries, the mystery of what sparked this odd phenomenon is finally solved.

>Dr. Jorge Pérez, an evolutionary biologist from the University of La Paz, and several companions, were exploring the Andes Mountains when they found a small valley, with no other animals or humans. Pérez noticed that the valley had what appeared to be a natural fountain, surrounded by two peaks of rock and silver snow.

>Pérez and the others then ventured further into the valley. “By the time we reached the top of one peak, the water looked blue, with some crystals on top,” said Pérez.

>Pérez and his friends were astonished to see the unicorn herd. These creatures could be seen from the air without having to move too much to see them – they were so close they could touch their horns.

>While examining these bizarre creatures the scientists discovered that the creatures also spoke some fairly regular English. Pérez stated, “We can see, for example, that they have a common ‘language,’ something like a dialect or dialectic.”

>Dr. Pérez believes that the unicorns may have originated in Argentina, where the animals were believed to be descendants of a lost race of people who lived there before the arrival of humans in those parts of South America.

>While their origins are still unclear, some believe that perhaps the creatures were created when a human and a unicorn met each other in a time before human civilization. According to Pérez, “In South America, such incidents seem to be quite common.”

>However, Pérez also pointed out that it is likely that the only way of knowing for sure if unicorns are indeed the descendants of a lost alien race is through DNA. “But they seem to be able to communicate in English quite well, which I believe is a sign of evolution, or at least a change in social organization,” said the scientist.

# GPT2 Text Generation

Here's the sample that's on the transformers website:

In [4]:
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [3]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

predicted_text

'Who was Jim Henson? Jim Henson was a man'

In [4]:
class TextGenerator:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # Load pre-trained model (weights)
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')

        # Set the model in evaluation mode to deactivate the DropOut modules
        # This is IMPORTANT to have reproducible results during evaluation!
        self.model.eval()
        # If you have a GPU, put everything on cuda
        self.model.to('cuda')

    
    def generate_word(self, start_text):
        """
        Generate one word (or sub-word, sometimes) to add onto some start text.
        The generated word will contain a leader space if appropriate.
        """
        # Encode text inputs
        indexed_tokens = self.tokenizer.encode(start_text)
        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])
        # Move the tokens to the GPU.
        tokens_tensor = tokens_tensor.to('cuda')
        

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word.
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        generated_word = self.tokenizer.decode(predicted_index)
        return generated_word
    
    def generate(self, start_text, word_count=7):
        text = start_text
        for _ in range(word_count):
            text += self.generate_word(text)
        return text
    
    def generate_sentence(self, start_text, sentence_count=1, up_to_count = None):
        if up_to_count:
            sentence_count = random.randint(1, up_to_count)
        text = start_text
        sentence = ''
        for _ in range(sentence_count):
            while (True):
                word = self.generate_word(text)
                text += word
                sentence += word
                if '.' in word:
                    break
        return sentence

gen = TextGenerator()

In [5]:
gen.generate_word("Can't stop now; I've travelled so")

' far'

In [6]:
gen.generate("Can't stop now; I've travelled",  10)

"Can't stop now; I've travelled a lot. I've been to a lot of"

In [29]:
text = gen.generate("My assignment was to wash the dishes.  I couldn't do that because",20)
text

"My assignment was to wash the dishes.  I couldn't do that because I wasn't sure if the water was warm enough.  If it was warm enough, the water"

In [8]:
text += "I feel terrible about that because"
gen.generate(text)

"My assignment was to wash the dishes.  I couldn't do that because I was too busy.               I feel terrible about that because I didn't do it. "

In [9]:
class BlameModeMe:
    my = 'my'
    i = 'I'
    me = 'me'

class BlameModeYou:
    my = 'your'
    i = 'you'
    me = 'you'

class BlameModeTeam:
    my = 'our'
    i = 'we'
    me = 'us'

class BlameModeTeamBlameShift:
    my = 'their'
    i = 'they'
    me = 'them'


In [10]:
INTRO_TEXT = [
    '[my] assignment was to',
    '[I] was supposed to',
    '[I] intended to',
    '[my] goal was to',
    '[my] dream, [my] destiny was to',
    '[I] had every intention to'
]

HOWEVER_TEXT = [
    "Sadly, that didn't work out because",
    "Unfortunately, there was a serious problem with that",
    "[I] couldn't do that because",
]

TASK_INTRO = [
    'Things were going pretty well until [I] got to the part where [I] needed to',
    '[I] hit a serious problem when [I] started to',
    '[I] hit a major roadblock when [I] began to',
    'It was going fine until [I] went to',
    'Things were going smoothly, but the problems started when I began to',
]

TASK_TRANSITION = [
    'The problem was',
    'What stopped [me] dead',
    'Where [I] ran into a roadblock was',
    "[I] couldn't finish this because",
    "[I] was unable to start this because",
    "[I] wanted to do this, but I couldn't because",
    "[I] started on that, but couldn't finish because",
]

EMOTION_TURNAROUND = [
    '[I] feel pretty sad about this because',
    "[I] hope you're not too disappointed and",
    "On the bright side,",
    "We shouldn't worry about it because",
    "But the real problem to focus on here is",
    "[I] am concerned about this because"
]

In [34]:
random.choice(HOWEVER_TEXT)

'Unfortunately, there was a serious problem with that'

In [35]:
'You know that [I] would never do that'.replace('[I]', 'we')

'You know that we would never do that'

In [14]:
class ExcuseSituation:
    def __init__(self, text_generator, assignment, tasks=[], is_team=False, blame_others=False):
        self.assignment = assignment
        self.tasks = tasks
        self.generator = text_generator
        if is_team:
            if blame_others:
                mode = BlameModeTeamBlameShift
            else:
                mode = BlameModeTeam
        else:
            if blame_others:
                mode = BlameModeYou
            else:
                mode = BlameModeMe
        self.mode = mode
    
    def generate_excuse(self):
        if random.random() < 0.5 or not self.tasks:
            text = self.generate_excuse_whole()
        else:
            text = self.generate_excuse_task()
        if random.random() < 0.5:
            text += random.choice(EMOTION_TURNAROUND)
            more = self.generator.generate_sentence(text)
            text = self._list_to_text([text, more])
        return text
        
    def generate_excuse_task(self):
        background = [random.choice(INTRO_TEXT), self.assignment, '.']
        background += [random.choice(TASK_INTRO), random.choice(self.tasks), '.']
        background += [random.choice(TASK_TRANSITION)]
        background_text = self._list_to_text(background)
        text = self.generator.generate_sentence(background_text, up_to_count=3)
        return self._prep_result(background_text + text)
        
    def generate_excuse_whole(self):
        # Lead-in text, setting up the situation.
        background = [random.choice(INTRO_TEXT), self.assignment, '.']
        background += [random.choice(HOWEVER_TEXT)]
        
        background_text = self._list_to_text(background)
        text = self.generator.generate_sentence(background_text)
        return self._prep_result(background_text + text)

    def generate_excuses(self, count=5):
        result = []
        for _ in range(count):
            result.append(self.generate_excuse())
        return result
            
    def _list_to_text(self, chunk_list):
        words = []
        for entry in chunk_list:
            entry = entry.replace('[I]', self.mode.i)
            entry = entry.replace('[my]', self.mode.my)
            entry = entry.replace('[me]', self.mode.me)
            words.append(entry)
        return ' '.join(words)
    
    def _prep_result(self, excuse_text):
        return excuse_text[0].upper() + excuse_text[1:] # Capitalize sentence
        

In [16]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses(count=3)

["I was supposed to prepare a nice dinner for you for Valentine's day . I couldn't do that because I was too busy with my work and my family.I hope you're not too disappointed and  I hope you're not too disappointed.",
 "My goal was to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that.But the real problem to focus on here is  that I was not able to get the recipe to work.",
 "My goal was to prepare a nice dinner for you for Valentine's day . I hit a major roadblock when I began to plan the menu . I wanted to do this, but I couldn't because I was too busy with work and my family. I had to make a decision. I wanted to make a meal that was good for you and that was good for me.We shouldn't worry about it because  we're all in this together."]

## Fight the Repetition

Sometimes there's a lot of repetition in the text. For example:
* I was supposed to prepare a nice dinner for you for Valentine's day . I couldn't do that because I was too busy with my work and my family. **I hope you're not too disappointed and  I hope you're not too disappointed.**

[GPT2 Issue: Repetition](https://github.com/huggingface/transformers/issues/1725).  Solution is to use *temperarture*.

A new TextGenerator with a temperature parameter to shake things up a bit:

In [37]:
import torch.nn.functional as F

class TextGenerator:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # Load pre-trained model (weights)
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')
        # Temperature adds randomness and unpredictability.
        self.temperature = random.choice([0.8, 0.8, 0.8, 0.9, 0.7])

        # Set the model in evaluation mode to deactivate the DropOut modules
        # This is IMPORTANT to have reproducible results during evaluation!
        self.model.eval()
        # If you have a GPU, put everything on cuda
        self.model.to('cuda')

    
    def generate_word(self, start_text):
        """
        Generate one word (or sub-word, sometimes) to add onto some start text.
        The generated word will contain a leader space if appropriate.
        """
        # Encode text inputs
        indexed_tokens = self.tokenizer.encode(start_text)
        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])
        # Move the tokens to the GPU.
        tokens_tensor = tokens_tensor.to('cuda')
        

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            # Use temperature to add more chaos into the selection of words.
            next_token_logits = outputs[0][:, -1, :] / self.temperature   # <--- Temperature
            next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1) # <-- Temperature

        # get the predicted next sub-word.
        generated_word = self.tokenizer.decode(next_token)
        return generated_word
    
    def generate(self, start_text, word_count=7):
        text = start_text
        for _ in range(word_count):
            text += self.generate_word(text)
        return text
    
    def generate_sentence(self, start_text, sentence_count=1, up_to_count = None):
        if up_to_count:
            sentence_count = random.randint(1, up_to_count)
        sentence_count=2 # TEMPORARY
        text = start_text
        sentence = ''
        for _ in range(sentence_count):
            while (True):
                word = self.generate_word(text)
                text += word
                sentence += word
                if '.' in word:
                    break
        return sentence

gen = TextGenerator()

In [39]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses(count=4)

["I intended to prepare a nice dinner for you for Valentine's day . It was going fine until I went to go to the grocery store to buy the ingredients . I started on that, but couldn't finish because I was having problems with the grocery store. It was really hard to get some ingredients on hand.",
 "I intended to prepare a nice dinner for you for Valentine's day . I hit a major roadblock when I began to plate the meal in an attractive way . I couldn't finish this because the waiter was too busy helping me out with a small bill. I apologize in advance for it being too late.",
 "I was supposed to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because you were late. I was hoping to get a fancy meal before Valentine's Day so I could go to bed and hang out with my friend.",
 "My dream, my destiny was to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that, and I was forced to wait till the next day to get a 

## Crazy stuff this came up with:
* My dream, my destiny was to prepare a nice dinner for you for Valentine\'s day . I hit a major roadblock when I began to plan the menu . What stopped me dead in my tracks?" What stopped me dead in my tracks?" "You know what, **I want you to live inside my house for a while**, so you can **enjoy my calm and wayward, hard-working, and true love**."
* I was supposed to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that dinner. My girlfriend thought I was **acting out as a man** so I had to be careful, but it turned out that there was a real problem.
* My assignment was to prepare a nice dinner for you for Valentine's day . Things were going pretty well until I got to the part where I needed to go to the grocery store to buy the ingredients . The problem was, I knew **there was already so much food in the fridge**. I had to look around to find the ingredients I needed and it was only over a month later when I got home that **I found the ingredient list for Trader Joe's frozen food**.
* I had every intention to prepare a nice dinner for you for Valentine's day . I hit a major roadblock when I began to plan the menu . What stopped me dead in my tracks was my own faraway world of extremities. **How could I conclude that food would be a primary way to trigger deep emotional cycles of desire?**
* My goal was to prepare a nice dinner for you for Valentine\'s day . I hit a serious problem when I started to go to the grocery store to buy the ingredients . The problem was that I had no idea how to prepare it. **I set aside my grocery money and said to myself, "where have I seen this** and how can I prove to myself how to prepare it with my own money?" **Now buy some flowers and you can actually have a good time.**
* My dream, my destiny was to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because **I had wasted my money and we both spent a lot**. Therefore, **I did all of my annual shopping for my daughter, and it has been a pleasure to watch her grow into a confident, confident young man.**
* I had every intention to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because my ass was still sore and I was already too sad and hungry to eat. So I decided to come up with an excuse to drive home, but then, having just finished my first breakfast, I ran into my friend, who just happened to be the new president of the United States.
*  **I was unable to start this because I had 42 different kinds of rice dishes. I was supposed to try each dish individually to see what would appeal to you the most.**
* Things we going smoothly, but the problems started when I began to plate the meal in an attractive way. I was unable to start this because I had to sit back for a couple of days, because with my Master's degree.\n\n I was so busy with waiting for the lunch that I wasn't doing the dishes properly. I feel pretty sad about this because  **I am very sick**, so it was hard for me to take care of this situation. **I was diagnosed with Cervus Neovis, which is a histolytic disease that affects bone health**.
* I intended to prepare a nice dinner for you for Valentine's day . It was going fine until I went to cook it up . I was unable to start this because I was starving but I had to before I could eat it.\n I made it in half.I feel pretty sad about this because I am going to make money and not have to take care of this. **I am gonna die soon, so I will be out of work.**

**Note**: If you're thinking about using GPT2 to generate text for children or business, you might want to put another model on there to classify if the output is appropriate.  Or maybe fine tune it a bit on your own corpus, and tune the temperature for your use case.

In [21]:
s = ExcuseSituation(gen, assignment="make sure the model generates text that is appropriate for the business, not outputting anything too offensive or racy", tasks=[
    'tune the chaos parameter',
    'classify the outputs with another model',
    'test the results on real children',
    'run it through a bad word filter',
])
s.generate_excuses(count=4)

['I had every intention to make sure the model generates text that is appropriate for the business, not outputting anything too offensive or racy . Things were going smoothly, but the problems started when I began to tune the chaos parameter . What stopped me dead was the fact that the model had an impressive range of measurements on the right side of the high-frequency recording, which might be interpreted as arbitrary, but what actually changed was the way the middle of the record was measured. The precision scale was different for the middle and the top end of the recording and for the different degrees of concave concave.',
 "I was supposed to make sure the model generates text that is appropriate for the business, not outputting anything too offensive or racy . Sadly, that didn't work out because the model did not include the real output of the other models. So, we had a problem in the one that was tested with 200 models.",
 'My dream, my destiny was to make sure the model generat

In [26]:
s = ExcuseSituation(gen, assignment='finish that report and analysis',
                    tasks=['talk to the customers', 'write up the customer feedback',
                          'run the numbers', 'do a detailed write-up of the analysis'])
s.generate_excuses(count=4)

['I was supposed to finish that report and analysis . It was going fine until I went to talk to the customers . I couldn\'t finish this because I couldn\'t talk to them, they wouldn\'t talk with me, which was the only way I could get my information."\n\n\n\n\n\n\n\n\n\n\n\n\n\n Now, for all the data you\'ve gathered, what\'s arguably the most interesting area is the sort of particularity you have in the varied cognitive abilities of people with ADHD (dementia).On the bright side,  you can also find more reasons to be interested in what cognitive abilities might explain something like this. The questions you asked were:\n\n \'Why does learning and language differ across different people with ADHD and schizophrenia?\'\n\nThey were very clear as to what the reasons were.',
 'My dream, my destiny was to finish that report and analysis . Things were going pretty well until I got to the part where I needed to do a detailed write-up of the analysis . What stopped me dead in my tracks was an u

In [27]:
goal = 'prepare for this technical talk'
tasks = ['consider various ideas for things to discuss', 'prepare a prototype',
         'create the presentation slides', 'write up the results', 'practice the presentation']
s = ExcuseSituation(gen, assignment=goal, tasks=tasks, is_team=False, blame_others=False)
s.generate_excuses(count=4)

["My dream, my destiny was to prepare for this technical talk . Things were going pretty well until I got to the part where I needed to practice the presentation . I couldn't finish this because of the lack of students, but I wanted to finish it and I was already in the main hall. I took my seat and I pretended to be a student, but people could see my pupils just like I did.I feel pretty sad about this because  this was my very first time in the media (and I only graduated 12 years ago). I didn't know many students from another major, so I didn't really know how to function or how to be helpful to people.",
 "I was supposed to prepare for this technical talk . Things were going smoothly, but the problems started when I began to practice the presentation . I wanted to do this, but I couldn't because of my age. I felt bad about it, but I didn't want to stay at home and let my Japanese friends call me and ask me questions.But the real problem to focus on here is  that I was human, Obama s

In [44]:
goal = 'put away the laundry'
tasks = ['getting it out of the dryer', 'folding it',
         'putting it on hangers', 'putting it in its specified location',
        'getting off the couch']
s = ExcuseSituation(gen, assignment=goal, tasks=tasks, is_team=False, blame_others=False)
s.generate_excuses(count=4)

["I had every intention to put away the laundry . I couldn't do that because I knew the crowd would be too packed. So I grabbed my belt and stepped in front of them.",
 'I intended to put away the laundry . Unfortunately, there was a serious problem with that it prevented me to get into the house.\n I had to cut the bathroom, and it is not installed correctly.',
 'My dream, my destiny was to put away the laundry . It was going fine until I went to folding it . I was unable to start this because it was getting too cold so I went to my neighbor\'s, and he was kind enough to let me go. When I got there, he had cleaned my bed and put it on its side, and I just sat there and thought "oh no, that\'s not going to work out right now, I\'m going to sleep in the closet, I\'m going to die in the closet," and then he ran out the door and, "I\'ll tell you what, I\'m going to get my name off the street and I\'m going to take care of my own life," and then I was gone, and that\'s when I started to re

In [45]:
goal = 'write a better summary for this document'
tasks = ['contemplate the most important ideas', 'determine the most efficient way to summarize the ideas',
]
s = ExcuseSituation(gen, assignment=goal, tasks=tasks, is_team=False, blame_others=False)
s.generate_excuses(count=4)

['My assignment was to write a better summary for this document . It was going fine until I went to determine the most efficient way to summarize the ideas . Where I ran into a roadblock was that of my editor. I was confused by the amount of time I had to explain to her what the idea of writing a better summary was.',
 "My goal was to write a better summary for this document . I hit a serious problem when I started to determine the most efficient way to summarize the ideas . I was unable to start this because I had already written a bit of things in the past. I think a lot of people use the memoizer as a way of organizing their code so all their comments will need to be written in an exact same way.But the real problem to focus on here is  that the outline of the document is covered so much in detail that you don't even know what the actual contents are. I decided to write just this portion of the document because I needed to expand this document a bit.",
 "I was supposed to write a be